In [ ]:
!pip install -qU langgraph langchain langchain-community duckduckgo-search langchain-openai python-dotenv ddgs

In [ ]:
# Set environment variables directly in Colab
import os
from getpass import getpass

# Ask for OpenRouter API key securely
openrouter_api_key = getpass('Enter your OpenRouter API key: ')
os.environ['OPENROUTER_API_KEY'] = openrouter_api_key

# Ask for Serper API key securely
serper_api_key = getpass('Enter your Serper API key: ')
os.environ['SERPER_API_KEY'] = serper_api_key

os.environ['MODEL'] = 'z-ai/glm-4.5-air:free'
os.environ['TEMPERATURE'] = '0.3'

# Also save to .env file for persistence
with open('.env', 'w') as f:
    f.write(f"OPENROUTER_API_KEY={openrouter_api_key}\n")
    f.write(f"SERPER_API_KEY={serper_api_key}\n")
    f.write(f"MODEL=z-ai/glm-4.5-air:free\n")
    f.write(f"TEMPERATURE=0.3\n")

Enter your OpenRouter API key: ··········
Enter your Serper API key: ··········


In [26]:
### Cell 3: Imports

from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, BaseMessage, ToolMessage
from langgraph.prebuilt import ToolNode


In [27]:
import smtplib
from getpass import getpass
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from email.mime.image import MIMEImage

def research_domain_tool(domain: str):
    """Researches a given domain using Google Serper."""
    print(f"--- Researching domain: {domain} ---")
    search = GoogleSerperAPIWrapper()
    research_result = search.run(f"information about {domain}")
    print(research_result)
    return research_result

def send_email_tool(to: str, subject: str, body: str):
    """Sends an email using SMTP."""
    # --- Office 365 Configuration ---
    sender_email = "shashikant.zarekar@gemengserv.com"
    smtp_server = "smtp.office365.com"
    smtp_port = 587
    # ------------------------------------

    # --- File Paths for Attachments ---
    # Ensure these files are uploaded to your Colab environment at these paths.
    logo_image_path = r"/content/GEM-Engserv-Pvt-Ltd-logo-updated.png"
    attachment_path = r"/content/GEM-Brochure_V0_13_10_2025.pdf"
    # ------------------------------------

    print("\n--- Preparing to send email ---")
    password = getpass(f"Enter password for {sender_email}: ")

    msg = MIMEMultipart()
    msg['From'] = f'Jaideep Singh - GEM Engserv<{sender_email}>'
    msg['To'] = to
    msg['Subject'] = subject

    # Attach the HTML message body generated by the agent
    msg.attach(MIMEText(body, 'html'))

    # Attach the logo image for embedding
    try:
        with open(logo_image_path, 'rb') as image_file:
            logo_image_mime = MIMEImage(image_file.read())
            logo_image_mime.add_header('Content-ID', '<logo>')
            msg.attach(logo_image_mime)
    except FileNotFoundError:
        print(f"Warning: Logo file not found at {logo_image_path}. Skipping logo.")

    # Attach the PDF brochure
    try:
        with open(attachment_path, 'rb') as file:
            attachment = MIMEApplication(file.read(), _subtype='pdf')
            attachment.add_header('Content-Disposition', 'attachment', filename="GEM Brochure.pdf")
            msg.attach(attachment)
    except FileNotFoundError:
        print(f"Warning: Attachment file not found at {attachment_path}. Skipping attachment.")

    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()
            server.ehlo()
            server.login(sender_email, password)
            server.send_message(msg)
        return "Email sent successfully!"
    except Exception as e:
        return f"Error sending email: {e}"

tools = [research_domain_tool, send_email_tool]

In [28]:
class AgentState(TypedDict):
    messages: Annotated[list[BaseMessage], operator.add]

In [29]:
# Instantiate the LLM to use OpenRouter
llm = ChatOpenAI(
    model=os.environ['MODEL'],
    temperature=float(os.environ['TEMPERATURE']),
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ['OPENROUTER_API_KEY']
)

# Bind the tools to the LLM
llm_with_tools = llm.bind_tools(tools)

# The agent node
def agent_node(state):
    """Calls the LLM to decide the next action."""
    response = llm_with_tools.invoke(state['messages'])
    return {"messages": [response]}

# The tool node
tool_node = ToolNode(tools)

# The router function
def router(state):
    """Routes the conversation to the correct node."""
    tool_calls = state['messages'][-1].tool_calls
    if not tool_calls:
        return "end"
    return "tools"

In [31]:
# Create a new graph
workflow = StateGraph(AgentState)

# Add the nodes to the graph
workflow.add_node("agent", agent_node)
workflow.add_node("tools", tool_node)

# Set the entry point of the graph
workflow.set_entry_point("agent")

# Add edges to define the flow
workflow.add_conditional_edges(
    "agent",
    router,
    {
        "tools": "tools",
        "end": END,
    },
)
workflow.add_edge("tools", "agent")


# Compile the graph into a runnable app
app = workflow.compile()

# Run the app
email_address = input("Please enter the recipient's email address: ")
domain = email_address.split('@')[1]

initial_prompt = f"""You are Jaideep Singh, handling Business Development at GEM Engserv having phone number as +91 8976167592, a top-tier construction project management consultancy (PMC). Your goal is to send a compelling, personalized sales email.

First, research the domain '{domain}' to understand the recipient's company.

Then, draft a persuasive email to {email_address}. The email body **must be in HTML format**.

Your email should:
1.  Introduce yourself and GEM Engserv, highlighting its expertise in construction PMC and engineering services.
2.  Use your research findings to tailor the message, showing how GEM Engserv can specifically benefit their company.
3.  Maintain a professional and compelling tone.
4.  Reference the company brochure, which is automatically attached to the email, for more details.
5.  Include the company logo in the signature by using `<img src="cid:logo">`.

Finally, use the send_email_tool to send your drafted HTML email."""

for chunk in app.stream({"messages": [HumanMessage(content=initial_prompt)]}):
    print(chunk)

Please enter the recipient's email address: shashikant.zarekar@gemengserv.com
{'agent': {'messages': [AIMessage(content='\n\n', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 770, 'prompt_tokens': 441, 'total_tokens': 1211, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'z-ai/glm-4.5-air:free', 'system_fingerprint': None, 'id': 'gen-1761721958-MGspgoErtbIHKtpLkkIz', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--7da2549f-b639-4b00-8881-adf2b3e28578-0', tool_calls=[{'name': 'research_domain_tool', 'args': {'domain': 'gemengserv.com'}, 'id': 'call_6689a7431e614822b3367084', 'type': 'tool_call'}], usage_metadata={'input_tokens': 441, 'output_tokens': 770, 'total_tokens': 1211, 'input_token_details': {}, 'output_token_details': {}})]}}
--- Researching domain: gemengserv.com ---
Specialized in project management consultancy services for the construction of Residential a